# Topic Extraction

In [1]:
import os
import logging
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF, LatentDirichletAllocation
from string import digits
import time
import re
import imgkit
import pandas as pd
logging.basicConfig(level=logging.DEBUG)

In [2]:
corpus_files = sorted(os.listdir('./corpus_per_president/'))

# Defining main functions
This data loader class will allow us to manage our data easily and focus on specific timelines or years. This is important since blindely looking at all data will incure in mixed topics that are related to all presidents throughout history. The usage of KMeans clustering along with term frequency–inverse document frequency feature extracture allows us to only focus on the important words and ignore the "useless" words such as "the" and "a" which do not provide essential information.

In [3]:
class LDA_DataLoader(object):    
    def __init__(self):
        self.data_folder = '/home/native/projects/NLP/finalProj/corpus_per_president/'
        self.years=sorted(os.listdir(self.data_folder))
        self.lda = LatentDirichletAllocation(n_components=10, max_iter=450, learning_method='online', learning_offset=5)
#         self.vectorizer = CountVectorizer(max_df=1, min_df=1, stop_words='english')
        self.vectorizer = TfidfVectorizer(max_df=0.5, min_df=1, stop_words='english',use_idf=True,norm='l2')

        pass
    
    def get_path(self,year):
        return self.data_folder + year + '/' 
    
    def get_filenames(self,year):
        return sorted(os.listdir(self.data_folder+'/'+year+'/'))
    
    def read_all_speeches(self):
#         self.corpus = {year:[] for year in self.years}
#         for year in self.years:
#             for speech in self.get_filenames(year):
#                 text = self.read_speech(speech,year).split()
#                 n=200
#                 chunks = [' '.join(text[i:i+n]) for i in range(0,len(text),n)]
#                 for chunk in chunks:
#                     self.corpus[year].append(chunk)
        self.corpus = {year:[] for year in self.years}
        for year in self.years:
            for speech in self.get_filenames(year):
                self.corpus[year].append(self.read_speech(speech,year))

    def read_speech(self, speech_filename,year):
        with open(self.data_folder +'/'+year+'/'+speech_filename, "rb") as file:
            text = file.read()
            if type(text) is not str:
                try:
                    text = text.decode("utf-8")
                except:
                    text = text.decode("iso-8859-1")
            #text = text.translate(None, digits)
            text = re.sub(r'[0-9]+', '', text)
            #text = text.replace("000", "")
            return text
        
    def cluster(self,year):
        X = self.vectorizer.fit_transform(dataloader.corpus[year])
        self.lda.fit(X)
        order_centroids = self.lda.components_.argsort()[:, ::-1]
        #print(order_centroids)
        terms = self.vectorizer.get_feature_names()
        key_topics=list()
        for ind in order_centroids[0,:10]:
            key_topics.append(terms[ind])
            #print("%s " % terms[ind], end='')
        return key_topics

In [4]:
class CorpusDataLoader(object):    
    def __init__(self):
        self.data_folder = '/home/native/projects/NLP/finalProj/corpus_per_president/'
        self.years=sorted(os.listdir(self.data_folder))
        self.km = KMeans(n_clusters=9, init='k-means++', max_iter=300, n_init=10,verbose=False)
        self.vectorizer = TfidfVectorizer(max_df=0.3, min_df=1, stop_words='english',use_idf=True,norm='l2')
        pass
    
    def get_path(self,year):
        return self.data_folder + year + '/' 
    
    def get_filenames(self,year):
        return sorted(os.listdir(self.data_folder+'/'+year+'/'))
    
    def read_all_speeches(self):
        self.corpus = {year:[] for year in self.years}
        for year in self.years:
            for speech in self.get_filenames(year):
                text = self.read_speech(speech,year).split()
                n=200
                chunks = [' '.join(text[i:i+n]) for i in range(0,len(text),n)]
                for chunk in chunks:
                    self.corpus[year].append(chunk)
#         self.corpus = {year:[] for year in self.years}
#         for year in self.years:
#             for speech in self.get_filenames(year):
#                 self.corpus[year].append(self.read_speech(speech,year))

    def read_speech(self, speech_filename,year):
        with open(self.data_folder +'/'+year+'/'+speech_filename, "rb") as file:
            text = file.read()
            if type(text) is not str:
                try:
                    text = text.decode("utf-8")
                except:
                    text = text.decode("iso-8859-1")
            text = text.replace("000", "")
            return text
        
    def cluster(self,year):
        X = self.vectorizer.fit_transform(dataloader.corpus[year])
        self.km.fit(X)
        order_centroids = self.km.cluster_centers_.argsort()[:, ::-1]
        terms = self.vectorizer.get_feature_names()
        key_topics=list()
        for ind in order_centroids[0,:10]:
            key_topics.append(terms[ind])
            #print("%s " % terms[ind], end='')
        return key_topics

In [5]:
years_list=[year[:9] for year in corpus_files]
presidents = [year[10:] for year in corpus_files]
df = pd.DataFrame(columns = years_list)
topics_by_year = {}

In [6]:
for file_name,year in zip(corpus_files,years_list):
    dataloader=CorpusDataLoader()
    dataloader.read_all_speeches()
    key_topics=dataloader.cluster(file_name)
    topics_by_year[year]=key_topics
    df[year]=key_topics


In [7]:
df=df.T
df['Presidents']=presidents

# Bringing data together
Putting everything into a table for a better visualization.

In [8]:
def highlight(data, color='yellow'):
    return 'background-color: %s' % color

df=df.style.applymap(highlight,subset=['Presidents'])
df

,0,1,2,3,4,5,6,7,8,9,Presidents
1789-1797,people,instituted,happiness,nations,success,ruler,supplications,country,providential,liberties,washington
1797-1801,glory,life,love,spirit,choice,fame,misfortune,classes,honor,humanity,adams
1801-1809,children,live,friends,people,men,visit,brothers,wish,brethren,war,jefferson
1809-1917,orders,british,repeal,commerce,french,britain,decrees,france,citizens,minister,madison
1817-1825,coast,report,force,fortifications,naval,department,engineers,war,protection,progress,monroe
1825-1829,years,lands,value,debt,public,10,small,nation,exports,prosperous,jqadams
1829-1837,treaty,commerce,france,relations,citizens,claims,friendly,minister,nations,intercourse,jackson
1837-1841,people,institutions,nations,principles,foreign,citizens,domestic,general,patriotism,prosperity,vanburen
1841-1845,mexico,texas,war,executive,annexation,state,people,nations,treaty,peace,tyler
1845-1849,public,banks,money,treasury,currency,paper,gold,bank,effect,credit,polk


In [9]:
html = df.render()
imgkit.from_string(html, 'KMeansClustering.png')

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

# LDA Approach
Comparison between LDA and Kmeans methods

In [10]:
df_lda = pd.DataFrame(columns = years_list)

In [11]:
for file_name,year in zip(corpus_files,years_list):
    dataloader=LDA_DataLoader()
    dataloader.read_all_speeches()
    key_topics=dataloader.cluster(file_name)
    topics_by_year[year]=key_topics
    df_lda[year]=key_topics

In [12]:
df_lda=df_lda.T
df_lda['Presidents']=presidents
def highlight(data, color='yellow'):
    return 'background-color: %s' % color

df_lda=df_lda.style.applymap(highlight,subset=['Presidents'])
df_lda

,0,1,2,3,4,5,6,7,8,9,Presidents
1789-1797,treaty,treaties,papers,end,advice,members,thirds,consent,making,respecting,washington
1797-1801,sufficient,executed,abandon,extensively,majestic,beneficial,considers,pliny,preliminary,coasts,adams
1801-1809,soundly,refitting,rom,imagined,world,refreshment,concerned,betrayed,following,connection,jefferson
1809-1917,church,episcopal,poor,identity,incorporating,protestant,corporate,polity,rightful,sundry,madison
1817-1825,explanations,ratified,catholic,grants,ratification,pecuniary,ratify,banks,meaning,alleged,monroe
1825-1829,feelings,affords,severe,unexaggerated,memorable,bremen,aspect,proportions,apprehended,mankind,jqadams
1829-1837,laden,greetings,achievement,beasts,notification,remorse,imperatively,excusable,awake,venal,jackson
1837-1841,apprised,proposed,declare,disproportioned,mainly,instruments,vainly,arrest,losses,poorer,vanburen
1841-1845,cotton,markets,courses,water,raw,grant,improvement,navigation,exportation,harbors,tyler
1845-1849,respectful,magnitude,live,partial,decline,consulted,oppressive,supposing,exertions,extract,polk


In [13]:
html_lda = df_lda.render()
imgkit.from_string(html_lda, 'LDA.png')

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True